In [1]:
import cvxpy as cp
import numpy as np

from RKHS import Gram

from functools import partial

In [3]:
def h(r,c,l):
    return c * np.exp(-(r**2) / l**2)

def k(x1,x2,c,l):
    small_sig = 1e-10 # This is needed for numerical stability.
    return h(np.linalg.norm(x1-x2+small_sig,axis = -1),c,l)

# The problem
Given reference points $a_{1:N}$, inducing $A = [k(a_i,a_j)]$.

The ultimate problem we attempt to solve is,

$$
		\max_{x_1,x_2,...,x_{T}} F(x_{1:T})\\
        ~||x_{t+1}-x_t||\leq D,~\forall t=1,...,T-1\\
		~ ||x_t-a_k||\leq R_{t,k},~\forall 1\leq t \leq T,~1\leq k\leq N
$$

where $R_{t,k}$ are pre-specified distance bounds to the reference points. 

The convex relaxation of the above we consider hereafter is
$$
    \max_{\Sigma,B}~\log\det(I+\Sigma/\sigma^2)\\
		s.t.~M=\begin{bmatrix}
    \Sigma & B\\
B^\top & A
\end{bmatrix}\succ 0\\
~\Sigma_{ii}=c,\forall i=1,2,...,T\\
~\Sigma_{i,i+1}\geq d,\forall i=1,...,T-1\\
~B_{ik}\geq b_{ik}=h(R_{i,k}),\forall i,k\\
(\text{Without admissibility constraints})
$$

Very often we want to impose the constraint that the initial location $x_1$ is fixed, this can be achieve by specifying the desired initial location as the value of $a_1$ and impose $||x_1-a_1||\leq 0$, or $b_{11} = h(0)$.

The optimal value of the convex relaxation is an upper bound on the ultimate problem, which could be loose.


In [36]:
T = 7

n = T
c = 1
l = 1 
step_size = 1

var = 0.01


d = h(step_size,c=c,l=l)

I = np.eye(n)


kernel = partial(k,c=c,l=l)

In [50]:
ref = np.array([[0,0]]) # A single point reference set.
R = np.array([[1]])
A = Gram(kernel,ref)

N = len(A)


b = np.ones((T,N))*h(R,c=c,l=l)
b[0,0] = h(0,c=c,l=l) # Initial location constraint.

In [51]:
# Define and solve the problem
B = cp.Variable((T,N))

S = cp.Variable((T,T),symmetric=True)

M = cp.vstack([cp.hstack([S,B]),
               cp.hstack([B.T,A])])


constraints = [M>>0]


constraints += [cp.diag(S)==c]

constraints += [S[i,i+1]>=d for i in range(0,T-2)]

constraints += [B>=b]



prob = cp.Problem(cp.Maximize(1/2*cp.log_det(np.eye(T)+ S/var)),constraints)

upper_bound = prob.solve()

In [52]:
upper_bound

15.578051640072232

In [ ]:
x